In [3]:
import numpy as np
import pandas as pd
import os
import pywikibot
import pickle

In [4]:
# Obtained from Temporal ranking Paper
#data = np.load("NASDAQ_wiki_relation.npy")
ticker_wikicode_map_df = pd.read_csv("NASDAQ_ticker_to_wiki.csv")

ticker_wikicode_map_nyse_df = pd.read_csv("NYSE_ticker_to_wiki.csv")


# Tickers that are selected for training the model
INDEX = "nasdaq100"

directory = "../../../data/" + INDEX + "/"

ticker_list = {}
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        ticker, name = filename.split("-")

        df = pd.read_csv(f)

        if df.shape[0] <= 2600: 
            continue 
        
        ticker_list[ticker] = [name]

INDEX = "sp500"

directory = "../../../data/" + INDEX + "/"

for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        ticker, name = filename.split("-")

        df = pd.read_csv(f)

        if df.shape[0] <= 2600: 
            continue 
        
        ticker_list[ticker] = [name]

print(len(ticker_list.keys()))
print(ticker_list.keys())

# Getting the WikiBase Id for the companies
# Some of the wanted tickers are not available.
wikicode_map = {}
for index, row in ticker_wikicode_map_df.iterrows():
    if row[0] in ticker_list:
        if row[1] == 'unknown':
            continue
        wikicode_map[row[0]] = row[1]

for index, row in ticker_wikicode_map_nyse_df.iterrows():
    if row[0] in ticker_list:
        if row[1] == 'unknown':
            continue
        wikicode_map[row[0]] = row[1]

print(len(wikicode_map.values()), len(ticker_list.values()))

# Obtained from Wikibase Query builder, using filter
# key 'part of' and value 's&p500
sp500_id = pd.read_csv("query.csv")

ticker_maps_mine = {}
for i in range(528):
    id = sp500_id.loc[i]['item'].split("/")[-1]
    print(sp500_id.loc[i])
    site = pywikibot.Site("wikidata", "wikidata")
    repo = site.data_repository()
    item = pywikibot.ItemPage(repo, id)
    item_dict = item.get()
    try:
        d = item_dict['claims']['P414']
        ticker = d[0].qualifiers['P249'][0].toJSON()['datavalue']['value']

        ticker_maps_mine[ticker] = id
    except:
        print("Not available")

# Even then, IDs for some tickers were not available, so I manually searched wikibase with company names
# and obtained the ID
my_map = {
    'MNST': 'Q1945295',
    'WBA': 'Q18712620',
    'EBAY': 'Q58024',
    'ENPH': 'Q17012234',
    'MTCH': 'Q23133268',
    'CHTR': 'Q2961234',
    'TMUS': 'Q3511885',
    'SIRI': 'Q3277465',
    'BKNG': 'Q18674747',
    'AMD': 'Q128896',
    'AVGO': 'Q4827416',
    'KDP': 'Q3116111',
    'SPLK': 'Q1835753',
    'CTRA': 'Q4035714',
    'PENN': 'Q7163201',
    'NEM': 'Q1785405',
    'GNRC': 'Q5531627',
    'TFC': 'Q795486',
    'PTC': 'Q1760641',
    'BKR': 'Q804353',
    'UAA': 'Q2031485',
    'SPGI': 'Q868587'    
}

ticker_id_map = {}
for ticker, val in ticker_list.items():
    if ticker in ticker_maps_mine:
        ticker_id_map[ticker] = ticker_maps_mine[ticker]
    elif ticker in my_map:
        ticker_id_map[ticker] = my_map[ticker]
    elif ticker in wikicode_map:
        ticker_id_map[ticker] = wikicode_map[ticker]
print(len(ticker_id_map.keys()))

with open('ticker_wiki_map.pkl', 'wb') as f:
    pickle.dump(ticker_id_map, f)

461
dict_keys(['SBUX', 'VRSK', 'ADBE', 'NVDA', 'AMGN', 'ISRG', 'KLAC', 'AAPL', 'REGN', 'BIDU', 'QCOM', 'XEL', 'MNST', 'CTSH', 'EA', 'WBA', 'ADSK', 'EBAY', 'ILMN', 'NTES', 'SWKS', 'SPLK', 'ADP', 'MU', 'AMZN', 'MCHP', 'NXPI', 'FTNT', 'SGEN', 'TXN', 'ENPH', 'DXCM', 'AZN', 'VRSN', 'MTCH', 'CHTR', 'AMAT', 'FISV', 'TMUS', 'MSFT', 'CTAS', 'IDXX', 'TSLA', 'MAR', 'AEP', 'NFLX', 'CDNS', 'ALGN', 'MELI', 'GILD', 'SIRI', 'EXC', 'CSCO', 'LULU', 'GOOGL', 'MRVL', 'ASML', 'BIIB', 'GOOG', 'BKNG', 'FAST', 'ORLY', 'ROST', 'VRTX', 'PCAR', 'CMCSA', 'PEP', 'ATVI', 'AMD', 'ANSS', 'AVGO', 'INTU', 'SNPS', 'COST', 'DLTR', 'PANW', 'HON', 'INTC', 'PAYX', 'CSX', 'LRCX', 'ADI', 'MDLZ', 'KDP', 'CPRT', 'ODFL', 'META', 'APH', 'PG', 'OMC', 'AMCR', 'KMX', 'STE', 'SPG', 'GE', 'TMO', 'PNR', 'EIX', 'SCHW', 'EQR', 'GIS', 'TRMB', 'FCX', 'UPS', 'FDX', 'SYY', 'COF', 'LEN', 'WELL', 'HSIC', 'BDX', 'BEN', 'LLY', 'VFC', 'WFC', 'WHR', 'UNP', 'FMC', 'GPC', 'ABT', 'IVZ', 'WAT', 'WEC', 'ROK', 'BRO', 'LNC', 'AME', 'TDY', 'DAL', 'CTRA', 

In [5]:
with open('ticker_wiki_map.pkl', 'rb') as f:
    ticker_wiki_map = pickle.load(f)

In [6]:
inverse_ticker_wiki_map = {ticker_wiki_map[k] : k for k in ticker_wiki_map}

In [7]:
# Relations to consider
relation_map = {
    'P127': 'Owned By - FO',
    'P155': 'Follows - FO',
    'P156': 'Followed By - FO',
    'P355': 'Subsidiary - FO',
    'P749': 'Parent Organisation - FO',
    'P31': 'Instance of',
    'P366': 'Use',
    'P452': 'Industry', 
    'P1056': 'Product or Material Produced',
    'P112': 'Founded By',
    'P169': 'CEO',
    'P113': 'Airline Hub',
    'P114': 'Airline Alliance',
    'P121': "Item Operated",
    'P1830': 'Owner of - FO',
    'P3320': 'Board Member',
    'P166': 'Award received',
    'P199': 'Business Division',
    'P361': 'Part of',
    'P400': 'Platform',
    'P2770': 'Source of Income',
    'P463': 'Member of',
    'P306': 'Operating System',
    'P1344': 'Participant of'
}

In [17]:
wiki_relations = {}

with open('wiki_relations_value.pkl', 'rb') as f:
    wiki_relations = pickle.load(f)

In [18]:
for relation, rel_label in relation_map.items():
    for ticker, wiki_id in ticker_wiki_map.items():
        if (ticker, relation) in wiki_relations:
            continue

        site = pywikibot.Site("wikidata", "wikidata")
        repo = site.data_repository()
        item_dict = pywikibot.ItemPage(repo, wiki_id).get()
        claims = item_dict['claims']

        try:
            value = claims[relation]
            for snaks in value:
                snak = snaks.toJSON()
                snak_value = snak['mainsnak']['datavalue']['value']['numeric-id']
                wiki_relations[(ticker, relation)] = wiki_relations.get((ticker, relation), []) + [snak_value]
            print(ticker, relation, rel_label, wiki_relations[(ticker, relation)])
        except:
            wiki_relations[(ticker, relation)] = None

ADBE P463 Member of [20947086, 858851, 37033, 67904121, 80656429, 852297, 82025113, 2934750, 492705, 2985457]
NVDA P463 Member of [15614763, 20947086, 858851, 1361045, 55524865, 67904121, 7389502, 16725802, 157754]
AAPL P463 Member of [20947086, 37033, 1361045, 7389502, 67904121, 79100838, 3616482, 2934750, 463681, 1008895, 1663561, 190835]
BIDU P463 Member of [858851, 37033, 2025721]
QCOM P463 Member of [1361045, 67904121, 15731702, 7389502, 3616482]
EA P463 Member of [55524865]
ADSK P463 Member of [858851, 67904121, 852297]
EBAY P463 Member of [858851, 63256596, 16848217, 15731702, 105272307]
SWKS P463 Member of [1361045]
SPLK P463 Member of [852297]
MU P463 Member of [858851, 16725802]
AMZN P463 Member of [20947086, 16848217, 37033, 7389502, 79100838, 15731702, 858851]
MCHP P463 Member of [858851, 1361045, 7389502, 3616482]
NXPI P463 Member of [858851, 1361045, 79100838, 15731702, 7389502, 3616482]
FTNT P463 Member of [1361045, 67904121]
TXN P463 Member of [858851, 1361045, 79100838

KeyboardInterrupt: 

In [19]:
with open('wiki_relations_value.pkl', 'wb') as f:
    pickle.dump(wiki_relations, f)

In [12]:
with open('wiki_relations_value.pkl', 'rb') as f:
    wiki_relations = pickle.load(f)

In [17]:
relation_knowledge_graph = []

In [18]:
for key, value in wiki_relations.items():
    if value is None:
        continue
    for ids in value:
        if 'Q'+str(ids) in inverse_ticker_wiki_map:
            print(key, inverse_ticker_wiki_map['Q'+str(ids)])
            relation_knowledge_graph.append((key[0], key[1], inverse_ticker_wiki_map['Q'+str(ids)]))

('NVDA', 'P127') BLK
('AAPL', 'P127') BLK
('MNST', 'P127') KO
('EBAY', 'P127') BLK
('AMZN', 'P127') BLK
('AZN', 'P127') BLK
('MSFT', 'P127') BLK
('MSFT', 'P127') STT
('MSFT', 'P127') BLK
('TSLA', 'P127') TROW
('TSLA', 'P127') TROW
('NFLX', 'P127') BLK
('NFLX', 'P127') MS
('NFLX', 'P127') BLK
('GILD', 'P127') BLK
('GOOGL', 'P127') BLK
('GOOGL', 'P127') BLK
('GOOG', 'P127') BLK
('GOOG', 'P127') BLK
('PEP', 'P127') BLK
('META', 'P127') BLK
('GE', 'P127') TROW
('GE', 'P127') BLK
('GE', 'P127') NTRS
('WFC', 'P127') BLK
('JPM', 'P127') BLK
('C', 'P127') BLK
('WU', 'P127') BLK
('WU', 'P127') STT
('MS', 'P127') STT
('MS', 'P127') BLK
('MS', 'P127') TROW
('CAT', 'P127') STT
('CAT', 'P127') BLK
('JNJ', 'P127') BLK
('JNJ', 'P127') STT
('MCD', 'P127') BLK
('MCD', 'P127') STT
('IBM', 'P127') STT
('VZ', 'P127') BLK
('XOM', 'P127') STT
('XOM', 'P127') BLK
('XOM', 'P127') BLK
('BAC', 'P127') BLK
('DRI', 'P127') GIS
('GLW', 'P127') BLK
('COP', 'P127') BLK
('PFE', 'P127') BLK
('PFE', 'P127') STT
('DHR',

In [20]:
second_order_relation_types = ['P452_P452', 'P127_P127', 'P355_P355', 'P355_P199', 
                  'P112_P112', 'P112_P127', 'P112_P169', 'P169_P112', 'P169_P127', 'P169_P169', 'P127_P112',
                  'P127_P169', 'P31_P366', 'P127_P3320', 'P463_P463', 'P1056_P306', 'P1056_P1056',
                  'P1056_P452', 'P31_P452', 'P199_P355', 'P452_P2770', 'P1344_P1344',
                  'P169_P3320', 'P452_P31', 'P452_P1056', 'P306_P1056', 'P121_P121', 'P31_P1056', 'P166_P166',
                  '2770_P452', 'P366_P31', 'P1056_P31', 'P400_P1056', 'P3320_P127', 'P3320_P169', 'P1056_P400',
                  'P114_P114', 'P121_P1056', 'P113_P113', 'P127_P749', 'P127_P355', 'P749_P127', 'P1830_P127', 
                  'P1830_P749', 'P355_P127', 'P155_P355', 'P155_P155', 'P355_P155', 'P127_P1830', 'P749_P1830',
                  'P1056_P121']
first_order_relation_types = ['P355', 'P155', 'P127', 'P156', 'P749', 'P1830']
print(len(relation_types))

57


In [31]:
value['mainsnak']

{'mainsnak': {'snaktype': 'value',
  'property': 'P127',
  'datatype': 'wikibase-item',
  'datavalue': {'value': {'entity-type': 'item', 'numeric-id': 849363},
   'type': 'wikibase-entityid'}},
 'type': 'statement',
 'id': 'Q182477$a4703b9e-4e87-0788-0eb3-69464f711ebc',
 'rank': 'normal',
 'qualifiers': {'P1107': [{'snaktype': 'value',
    'property': 'P1107',
    'datatype': 'quantity',
    'datavalue': {'value': {'amount': '+0.0768',
      'upperBound': None,
      'lowerBound': None,
      'unit': '1'},
     'type': 'quantity'},
    'hash': '68ac81f5061b94d57c8533c2711b89da24479e81'}],
  'P585': [{'snaktype': 'value',
    'property': 'P585',
    'datatype': 'time',
    'datavalue': {'value': {'time': '+00000002020-03-22T00:00:00Z',
      'precision': 11,
      'after': 0,
      'before': 0,
      'timezone': 0,
      'calendarmodel': 'http://www.wikidata.org/entity/Q1985727'},
     'type': 'time'},
    'hash': '51ede5fdbff3e5aa3710887eb267d8e36caf8c62'}]},
 'qualifiers-order': ['P11

In [13]:
#Get the item
site = pywikibot.Site("wikidata", "wikidata")
repo = site.data_repository()
item = pywikibot.ItemPage(repo, "Q3884")

In [23]:
item_dict.keys()

dict_keys(['labels', 'descriptions', 'aliases', 'claims', 'sitelinks'])

In [24]:
for k, v in item_dict['claims'].items():
    print(k)
    break

P1297


In [25]:
item_dict['claims']['P127']

[Claim.fromJSON(DataSite("wikidata", "wikidata"), {'mainsnak': {'snaktype': 'value', 'property': 'P127', 'datatype': 'wikibase-item', 'datavalue': {'value': {'entity-type': 'item', 'numeric-id': 312556}, 'type': 'wikibase-entityid'}}, 'type': 'statement', 'id': 'Q3884$047da791-4711-4e0d-2eae-a6c87c11785d', 'rank': 'preferred', 'qualifiers': {'P1107': [{'snaktype': 'value', 'property': 'P1107', 'datatype': 'quantity', 'datavalue': {'value': {'amount': '+0.16', 'upperBound': None, 'lowerBound': None, 'unit': '1'}, 'type': 'quantity'}, 'hash': 'b018b2314853700360425a21c6db5e26b3b84fb7'}], 'P585': [{'snaktype': 'value', 'property': 'P585', 'datatype': 'time', 'datavalue': {'value': {'time': '+00000002019-02-25T00:00:00Z', 'precision': 11, 'after': 0, 'before': 0, 'timezone': 0, 'calendarmodel': 'http://www.wikidata.org/entity/Q1985727'}, 'type': 'time'}, 'hash': '88f279345ccb2b6d3c482c5178cb8bbf3ac7dd72'}]}, 'qualifiers-order': ['P1107', 'P585'], 'references': [{'snaks': {'P854': [{'snakty

In [17]:
import json

x = open('NASDAQ_connections.json', "r")
# parse x:
y = json.loads(x.read())

In [20]:
y['Q5969562']

{'Q20022777': [['P17', 'P17'], ['P31', 'P31'], ['P414', 'P414']],
 'Q6648755': [['P31', 'P31'], ['P414', 'P414']],
 'Q470517': [['P17', 'P17'], ['P31', 'P31'], ['P414', 'P414']],
 'Q4816705': [['P17', 'P17'], ['P414', 'P414']],
 'Q5969568': [['P17', 'P17'], ['P31', 'P31'], ['P414', 'P414']],
 'Q1045842': [['P414', 'P414']],
 'Q1544413': [['P17', 'P17'], ['P31', 'P31'], ['P414', 'P414']],
 'Q5001347': [['P17', 'P17'], ['P31', 'P31'], ['P414', 'P414']],
 'Q18156655': [['P17', 'P17'], ['P414', 'P414']],
 'Q17183462': [['P17', 'P17'], ['P31', 'P31'], ['P414', 'P414']],
 'Q1071853': [['P17', 'P17'], ['P414', 'P414']],
 'Q6702957': [['P31', 'P31'], ['P414', 'P414']],
 'Q297879': [['P31', 'P31'], ['P414', 'P414']],
 'Q431454': [['P31', 'P31'], ['P414', 'P414']],
 'Q946080': [['P31', 'P31'], ['P414', 'P414']],
 'Q1325369': [['P31', 'P31'], ['P414', 'P414']],
 'Q20800404': [['P17', 'P17'], ['P31', 'P31'], ['P414', 'P414']],
 'Q6843476': [['P31', 'P31'], ['P414', 'P414']],
 'Q1665429': [['P17', 

Number not got:  0
